In [1]:
import pandas as pd
import numpy as np

In [2]:
migflow2015_county_nodeid_dist = pd.read_csv("./NTDS_Data/migflow2015_county_nodeid_dist.csv")
migflow2015_county_nodeid_dist.head()

,ori_id,des_id,ori_county,des_county,returns,exemptions,AGI,dist_km
0,1366,1758,1051,1001,387,895,17374,46.761904
1,336,1758,1101,1001,352,741,13267,53.795844
2,641,1758,1021,1001,68,147,2111,35.517244
3,1114,1758,1047,1001,57,132,2253,49.334907
4,636,1758,1073,1001,45,81,1974,115.791496


In [5]:
migflow2015_county_nodeid_dist = migflow2015_county_nodeid_dist.drop(columns=["ori_county", "des_county","dist_km"])
migflow2015_county_nodeid_dist.head()

,ori_id,des_id,returns,exemptions,AGI
0,1366,1758,387,895,17374
1,336,1758,352,741,13267
2,641,1758,68,147,2111
3,1114,1758,57,132,2253
4,636,1758,45,81,1974


In [6]:
# graph just from migration

In [ ]:
# graph, distance 

In [ ]:
#